### Using an external Rust library to speed up lon, lat to [BNG](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) conversion

In [1]:
import numpy as np
import pandas as pd
import math
from ctypes import cdll, c_float, Structure, ARRAY, POINTER, c_int32, c_uint32, c_size_t, c_void_p, cast
from sys import platform
from bng import bng
import pyproj

In [2]:
if platform == "darwin":
    ext = "dylib"
else:
    ext = "so"

### Setting up the Rust library. See [here](https://github.com/alexcrichton/rust-ffi-examples/tree/master/python-to-rust) for more

Ensure you've built your Rust library using `cargo build --release`, or the next step will fail.

The boilerplate below can easily be hidden in a wrapper function – it's just here to demonstrate how to call into a shared Rust lib using FFI.

In [3]:
lib = cdll.LoadLibrary('target/release/liblonlat_bng.' + ext)


class FFITuple(Structure):
    _fields_ = [("a", c_uint32),
                ("b", c_uint32)]

class FFIArray(Structure):
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    # Allow implicit conversions from a sequence of 32-bit unsigned
    # integers.
    @classmethod
    def from_param(cls, seq):
        return seq if isinstance(seq, cls) else cls(seq)

    # Wrap sequence of values. You can specify another type besides a
    # 32-bit unsigned integer.
    def __init__(self, seq, data_type = c_float):
        array_type = data_type * len(seq)
        raw_seq = array_type(*seq)
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)

# A conversion function that cleans up the result value to make it
# nicer to consume.
def void_array_to_tuple_list(array, _func, _args):
    return cast(array.data, POINTER(FFITuple * array.len))[0]

# Single-threaded
convert_vec = lib.convert_vec_c
convert_vec.argtypes = (FFIArray, FFIArray)
convert_vec.restype = FFIArray
convert_vec.errcheck = void_array_to_tuple_list

# Multi-threaded
convert_vec_threaded = lib.convert_to_bng
convert_vec_threaded.argtypes = (FFIArray, FFIArray)
convert_vec_threaded.restype = FFIArray
convert_vec_threaded.errcheck = void_array_to_tuple_list


def convertbng(lons, lats):
    """ Multi-threaded wrapper """
    return [(i.a, i.b) for i in iter(convert_vec(lons, lats))]

def convertbng_threaded(lons, lats):
    """ Multi-threaded wrapper """
    return [(i.a, i.b) for i in iter(convert_vec_threaded(lons, lats))]

## Simple test of average conversion speed, Python version

In [4]:
%%timeit -r 10
bng(51.44533267, -0.32824866)

100000 loops, best of 10: 19.2 µs per loop


In [8]:
%%timeit -r 10
convertbng_list([-0.32824866], [51.44533267])

The slowest run took 4.01 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 10: 79.3 µs per loop


The threaded version's slower. Threads have overhead!

## A slightly more realistic test: 10MM random points within the UK

In [5]:
# UK bounding box
N = 55.811741
E = 1.768960
S = 49.871159
W = -6.379880

bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

lon_ls = list(np.random.uniform(W, E, [1000000]))
lat_ls = list(np.random.uniform(N, S, [1000000]))

### Pure Python

In [9]:
%%timeit
[bng(lat, lon) for lat, lon in zip(lat_ls, lon_ls)]

1 loops, best of 3: 5.26 s per loop


### Pyproj

In [8]:
%%timeit -r 15
proj_res = zip(*pyproj.transform(wgs84, bng, lon_ls, lat_ls))

1 loops, best of 5: 1.01 s per loop


### Single-threaded Rust

In [9]:
%%timeit -r 15
convertbng(lon_ls, lat_ls)

1 loops, best of 5: 2.01 s per loop


### Multithreaded Rust

In [6]:
%%timeit -r 15
convertbng_threaded(lon_ls, lat_ls)

1 loops, best of 15: 1.59 s per loop


## Pyproj is now only ~65% faster, multithreaded Rust version is 5x faster than pure Python